In [3]:
import tensorflow as tf
from matplotlib import pyplot
import numpy as np
import os
from PIL import Image

tf.enable_eager_execution()

In [4]:
def wrap_int64(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))
def wrap_bytes(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))


In [5]:
channels=3
image_size=53
train_ratio=0.8
val_ratio=0.1

tf.Tensor(3, shape=(), dtype=int32)


In [4]:
image_path="/home/justdial/test_tfrecord/"
label=0
labels=[]
train=[]
val=[]
test=[]
for cat in os.listdir(image_path):
    print("label for  {} is {} ".format(cat,label))
    images=os.listdir(os.path.join(image_path,cat))
    total_images=len(images)
    print(total_images)
    num_train=int(total_images*train_ratio)
    num_val=int(total_images*val_ratio)
    num_test=int(total_images-(num_train+num_val))
    print("num_train={}".format(num_train))
    print("num_val={}".format(num_val))
    print("num_test={}".format(num_test))
    for i,img in enumerate(images):
        if(i<=num_train):
            train.append([os.path.join(os.path.join(image_path,cat),img),label])
        elif(i>num_train and i<=num_train+num_val):
            val.append([os.path.join(os.path.join(image_path,cat),img),label])
        elif(i<=total_images and i>num_train+num_val):
            test.append([os.path.join(os.path.join(image_path,cat),img),label])
            
    label +=1
#     labels.append(label)

#print(train)
print(len(train))
print(len(val))
print(len(test))

label for  reception is 0 
383
num_train=306
num_val=38
num_test=39
label for  conference is 1 
218
num_train=174
num_val=21
num_test=23
482
59
60


In [5]:
out_path="/home/justdial/tfrecordssss/tfrecordtrain"
writer1=tf.python_io.TFRecordWriter(out_path)
writer2=tf.python_io.TFRecordWriter("/home/justdial/tfrecordssss/tfrecordval")
writer3=tf.python_io.TFRecordWriter("/home/justdial/tfrecordssss/tfrecordtest")

In [6]:
def get_binary_image(img_path):
    try:
        img =Image.open(img_path)
        img = img.resize((image_size,image_size),Image.ANTIALIAS)
        img = img.tobytes()
        #print(len(img))
        if(len(img)==image_size*image_size*channels):
            #print(img)
            return img
    except:
        print('{} caNT BE CONVERTED'.format(img_path))
        pass

In [7]:
def create_tf_Record(image_bytes,label):
    data={'image':wrap_bytes(image_bytes),
         'label':wrap_int64(label)}
    feature=tf.train.Features(feature=data)
    example=tf.train.Example(features=feature)
    serialized=example.SerializeToString()
    return serialized

In [8]:
#created a tfrecord with train images
count = 0
for img in train:
    try:
        #print(img)
        img_path1=img[0]
        #print(img_path1)
        label1=img[1]
        #print(label1)
        image_bytes=get_binary_image(img_path1)
        #print(image_bytes)
        #print(label1)
        #print(count)
        serialized=create_tf_Record(image_bytes,label1)
        writer1.write(serialized)
    except:
        pass
    #count+=1
writer1.close()

In [9]:
for img in val:
    try:
        #print(img)
        img_path1=img[0]
        #print(img_path1)
        label1=img[1]
        #print(label1)
        image_bytes=get_binary_image(img_path1)
        #print(image_bytes)
        #print(label1)
        #print(count)
        serialized=create_tf_Record(image_bytes,label1)
        writer2.write(serialized)
    except:
        pass
    #count+=1
writer2.close()

In [10]:
for img in test:
    try:
        #print(img)
        img_path1=img[0]
        #print(img_path1)
        label1=img[1]
        #print(label1)
        image_bytes=get_binary_image(img_path1)
        #print(image_bytes)
        #print(label1)
        #print(count)
        serialized=create_tf_Record(image_bytes,label1)
        writer3.write(serialized)
    except:
        pass
    #count+=1
writer3.close()

In [6]:
def parse(serialized):
    features={
        'image':tf.FixedLenFeature([],tf.string),
        'label':tf.FixedLenFeature([],tf.int64)
    }
    parsed_example=tf.parse_single_example(serialized=serialized,
                                          features=features)
    raw_image=parsed_example['image']
    image=tf.decode_raw(raw_image,tf.uint8)
    
    image = tf.reshape(image,shape=[image_size,image_size,channels])

    image=tf.cast(image,tf.float64)    
    label=parsed_example['label']
    return image,label

In [ ]:
def parse1(serialized):
    features={
        'image':tf.FixedLenFeature([],tf.string),
        'label':tf.FixedLenFeature([],tf.int64)
    }
    parsed_example=tf.parse_single_example(serialized=serialized,
                                          features=features)
    raw_image=parsed_example['image']
    image=tf.decode_raw(raw_image,tf.uint8)
    
    image = tf.reshape(image,shape=[image_size,image_size,channels])

    image=tf.cast(image,tf.float64)
    return image

In [22]:
def input_fn():
    dataset=tf.data.TFRecordDataset(filenames='/home/justdial/tfrecordssss/tfrecordtrain')
    dataset=dataset.map(parse)
    #     if train:
    #         dataset=dataset.shuffle(buffer_size=buffer_size)
    #         num_repeat=None
    #     else:
    #         num_repeat=1
    # #     dataset=dataset.repeat(num_repeat)
    dataset=dataset.batch(32)
    #     iterator=dataset.make_one_shot_iterator()
    iterator= tf.data.Iterator.from_structure(output_types = dataset.output_types, output_shapes = dataset.output_shapes)    
    #     img_batch,labels_batch=iterator.get_next()
    iterator2 = tf.contrib.eager.Iterator(dataset)
#     for x in iterator2(dataset):
#         print(x)
#     init = iterator.make_initializer(dataset)
#     init
    img_batch, Y = iterator2.get_next()
    X = {'image': img_batch} 
#     print (X,Y)
#     # with tf.Session() as sess:
#     #     sess.run(init)
    return X,Y


        #     Y=labels_batch

TypeError: __init__() missing 1 required positional argument: 'dataset'

In [6]:
init

In [12]:

def input_fn(filenames,train,sess, batch_size=32,buffer_size=2048):
        X={'image':img_batch}

    return X,Y

In [12]:
dataset=tf.data.TFRecordDataset(filenames='/home/justdial/tfrecordssss/tfrecordtrain')
dataset=dataset.map(parse)
iterator2 = tf.contrib.eager.Iterator(dataset)

iterator1=dataset.make_one_shot_iterator()
#     if train:
#         dataset=dataset.shuffle(buffer_size=buffer_size)
#         num_repeat=None
#     else:
#         num_repeat=1
# #     dataset=dataset.repeat(num_repeat)
#     dataset=dataset.batch(batch_size)
#     iterator=dataset.make_one_shot_iterator()


W0711 13:24:51.035992 140116951471872 deprecation.py:323] From <ipython-input-12-bcc728d01036>:5: DatasetV1.make_one_shot_iterator (from tensorflow.python.data.ops.dataset_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_one_shot_iterator(dataset)`.


In [21]:
print (iterator2.get_next)

AttributeError: 'function' object has no attribute 'value'

In [15]:
for x in iterator2:
    print (x)

(<tf.Tensor: id=37, shape=(53, 53, 3), dtype=float64, numpy=
array([[[254., 228., 154.],
        [255., 228., 160.],
        [254., 231., 161.],
        ...,
        [253., 253., 254.],
        [253., 253., 249.],
        [252., 253., 238.]],

       [[254., 224., 151.],
        [255., 225., 157.],
        [254., 228., 159.],
        ...,
        [253., 254., 251.],
        [251., 254., 241.],
        [252., 251., 222.]],

       [[253., 222., 148.],
        [253., 225., 149.],
        [254., 227., 153.],
        ...,
        [253., 252., 235.],
        [251., 249., 218.],
        [250., 243., 192.]],

       ...,

       [[248., 175.,  78.],
        [251., 178.,  84.],
        [251., 177.,  82.],
        ...,
        [141.,  90.,  33.],
        [147.,  94.,  35.],
        [150.,  95.,  32.]],

       [[251., 177.,  87.],
        [249., 176.,  86.],
        [247., 172.,  77.],
        ...,
        [136.,  86.,  24.],
        [138.,  87.,  26.],
        [139.,  90.,  30.]],

       [[24

(<tf.Tensor: id=669, shape=(53, 53, 3), dtype=float64, numpy=
array([[[145., 124., 106.],
        [155., 134., 117.],
        [130., 115., 113.],
        ...,
        [159., 138., 122.],
        [142., 120., 107.],
        [ 96.,  80.,  72.]],

       [[149., 126., 109.],
        [164., 140., 118.],
        [175., 153., 131.],
        ...,
        [155., 132., 116.],
        [122., 104.,  91.],
        [ 36.,  32.,  30.]],

       [[156., 132., 112.],
        [167., 141., 119.],
        [176., 153., 130.],
        ...,
        [144., 121., 106.],
        [ 60.,  51.,  47.],
        [  3.,   5.,   7.]],

       ...,

       [[215., 195., 171.],
        [221., 202., 178.],
        [225., 205., 182.],
        ...,
        [221., 215., 215.],
        [211., 205., 206.],
        [207., 201., 203.]],

       [[210., 190., 166.],
        [218., 199., 175.],
        [227., 208., 184.],
        ...,
        [218., 212., 212.],
        [210., 204., 205.],
        [203., 197., 199.]],

       [[2

(<tf.Tensor: id=1221, shape=(53, 53, 3), dtype=float64, numpy=
array([[[ 58.,  42.,  12.],
        [ 61.,  45.,  12.],
        [ 65.,  51.,  11.],
        ...,
        [ 19.,   7.,   8.],
        [ 18.,   9.,   5.],
        [ 17.,   7.,   4.]],

       [[ 57.,  44.,  13.],
        [ 61.,  49.,  14.],
        [ 65.,  54.,  13.],
        ...,
        [ 19.,   7.,   9.],
        [ 18.,   7.,   8.],
        [ 17.,   7.,   8.]],

       [[ 56.,  48.,  16.],
        [ 59.,  53.,  16.],
        [ 62.,  54.,  12.],
        ...,
        [ 19.,   8.,  11.],
        [ 18.,   8.,  13.],
        [ 18.,   9.,  13.]],

       ...,

       [[ 91.,  85., 117.],
        [ 99.,  97., 125.],
        [ 75.,  79.,  90.],
        ...,
        [137., 145., 202.],
        [123., 142., 193.],
        [115., 135., 186.]],

       [[ 89.,  86., 117.],
        [ 95.,  93., 120.],
        [ 87.,  85., 106.],
        ...,
        [125., 137., 192.],
        [116., 135., 185.],
        [105., 124., 178.]],

       [[

In [75]:
iterator1.output_shapes

W0711 12:50:57.262423 140608120473344 deprecation.py:323] From <ipython-input-75-3f285bafe2f6>:1: Iterator.output_shapes (from tensorflow.python.data.ops.iterator_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.compat.v1.data.get_output_shapes(iterator)`.


(TensorShape([Dimension(53), Dimension(53), Dimension(3)]), TensorShape([]))

In [65]:
a =iterator.get_next()

In [66]:
a

(<tf.Tensor 'IteratorGetNext:0' shape=(53, 53, 3) dtype=float64>,
 <tf.Tensor 'IteratorGetNext:1' shape=() dtype=int64>)

In [12]:
def train_inp_fun():
     
    return X,Y

In [71]:
def val_inp_fun():
    return input_fn(filenames="/home/justdial/tfrecordssss/tfrecordval",train=False)

In [72]:
def test_inp_fun():
    return input_fn(filenames="/home/justdial/tfrecordssss/tfrecordtest",train=False)

In [23]:
feature_image=tf.feature_column.numeric_column("image",shape=[53,53,3])
feature_columns=[feature_image]
num_hidden_units=[30,20,10]
# with tf.Session() as sess:
    
model = tf.estimator.DNNClassifier(feature_columns=feature_columns,
                                hidden_units=num_hidden_units,
                                activation_fn=tf.nn.relu,
                                n_classes=2)
model.train(input_fn=input_fn,steps=20000000)

W0711 13:27:56.715963 140116951471872 estimator.py:1811] Using temporary folder as model directory: /tmp/tmpht7tgo8r
W0711 13:27:56.720969 140116951471872 deprecation.py:323] From /home/justdial/miniconda3/lib/python3.7/site-packages/tensorflow/python/training/training_util.py:236: Variable.initialized_value (from tensorflow.python.ops.variables) is deprecated and will be removed in a future version.
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
W0711 13:27:56.742305 140116951471872 deprecation.py:323] From <ipython-input-22-fc1341f46749>:12: DatasetV1.output_types (from tensorflow.python.data.ops.dataset_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.compat.v1.data.get_output_types(dataset)`.
W0711 13:27:56.742886 140116951471872 deprecation.py:323] From <ipython-input-22-fc1341f46749>:12: DatasetV1.output_shapes (from tensorflow.pyth

NotFoundError: Resource AnonymousIterator/AnonymousIterator2/N10tensorflow4data16IteratorResourceE does not exist.
	 [[node IteratorGetNext (defined at <ipython-input-22-fc1341f46749>:19) ]]

Errors may have originated from an input operation.
Input Source operations connected to node IteratorGetNext:
 AnonymousIteratorV2 (defined at <ipython-input-22-fc1341f46749>:14)

Original stack trace for 'IteratorGetNext':
  File "/home/justdial/miniconda3/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/justdial/miniconda3/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/justdial/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/justdial/miniconda3/lib/python3.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/justdial/miniconda3/lib/python3.7/site-packages/ipykernel/kernel

In [57]:
feature_image=tf.feature_column.numeric_column("image",shape=[53,53,3])

In [58]:
feature_image

NumericColumn(key='image', shape=(53, 53, 3), default_value=None, dtype=tf.float32, normalizer_fn=None)

In [ ]:
def predict_inp_func1():
    
    dataset=tf.data.TFRecordDataset(filenames="/home/justdial/tfrecordssss/tfrecordval")
    dataset=dataset.map(parse1)
    dataset=dataset.batch(batch_size=1)
    iterator=dataset.make_one_shot_iterator()
    img_batch=iterator.get_next()
    X={'image':img_batch}
    return X


In [ ]:
def extractimages():
    #os.mkdir("/home/justdial/test_tfrecord/extractedtest")
    dataset=tf.data.TFRecordDataset(filenames="/home/justdial/tfrecordssss/tfrecordtest")
    dataset=dataset.map(parse)
    iterator=dataset.make_one_shot_iterator()
    img_batch=iterator.get_next()
    print(img_batch)
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        try:
            while True:
                print("here2")
                image_data=sess.run(img_batch)
                print("here")
#                 if(len(image_data[0])==(image_size*image_size*channels)):
                if not np.array_equal(image_data[0].shape, image_data[3]):
                    print('{} not decoded properly'.format(image_data[0]))
                    continue
                savepath=os.path.join("/home/justdial/test_tfrecord/extractedtest",image_data[2])
                mpimg.imsave(save_path, image_data[0])
#                 else:
#                     print('{} not decoded properly'.format(image_data[0]))
        except:
            print("some error")
            pass

In [ ]:
# def load_image( infilename ) :
    
#     img = Image.open( infilename )
#     img.load()
#     data = np.asarray( img, dtype="int32" )
#     return data
print(tf.__version__)
print(dir(tf.feature_column))

In [ ]:

iamge = load_image("/home/justdial/test_tfrecord/extractedtest/109297.JPG")

In [ ]:
some_images=[]
def predict_images():
    for i in os.listdir("/home/justdial/extractedtest"):
        img = Image.open(os.path.join("/home/justdial/extractedtest/",i))
        img = img.resize((image_size,image_size),Image.ANTIALIAS)
        #print(img)
#         img = np.array(img)
#         img = img.reshape(53,53,3)
#         print(img.shape)
#         some_images.append(load_image(os.path.join("/home/justdial/test_tfrecord/extractedtest/",i)))
        some_images1 =np.append(some_images,img)
    print(some_images1.shape)
    print(len(some_images1))

    some_image = np.array(some_images1)
    print(some_image.shape)

    predict_input_fn=tf.estimator.inputs.numpy_input_fn(x={"image":some_image.astype(np.float32)},
                                                        num_epochs=1,shuffle=False)
    
    result = model.evaluate(input_fn=val_inp_fun)
    print(result)
#     predictions =model.predict(input_fn=predict_input_fn)
#     return predictions

In [ ]:
predictions=model.predict(input_fn=predict_inp_func1)
print((predictions))

In [ ]:
predictions=predict_images()
#print('Predictions: {}' .format(list(predictions#)))

In [ ]:
print(list(predictions))

In [ ]:
cls = [p['classes'] for p in predictions]
print(cls)
cls_pred=np.array(cls,dtype="int").squeeze()
print(cls_pred)

In [ ]:
for i in predictions:
    print(i)